In [13]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import gc
import scipy

import sys
import os
import datetime
from random import randrange

import time

from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

# to make this notebook's output stable across runs
np.random.seed(42)

In [14]:
class Clustering():

    print("Entry")
    def pre_period_func (self,pre_data, matched_list, test_unmatched_list, control_non_replacement):

        #Total distict test_control_flag
        pre_test_check = len(pd.unique(pre_data['test_control_flag']))

        cols = ["dr_visit",
                "rx_tzd_sfu",
                "rx_ddp4_sglt2",
                "rx_glp1",
                "rx_metformin",
                "rx_other"]

        if pre_test_check == 1: #Only test patients in the data
            pre_test_check_value = pre_data.test_control_flag.unique()
            if pre_test_check_value == ['test']: #Add T to the unmatched list to match with Patient table
                T_only = pre_data[["patient_id", "combination_key"]]
                T_only["clusters"] = 0
                test_unmatched_list = test_unmatched_list.append(T_only)    

        if pre_test_check == 2:
            #Remove control patients who have already been matched
            #Select Control patients from pre period group who are not in control_non_replacement
            
            #########What is control non replacement  and test non replacement##########how is this join working
            pre_data = pd.merge(pre_data, control_non_replacement, on=["patient_id"], how="outer", 
                                indicator=True).query('_merge=="left_only"') #Left Excluding join

            #Separating Test data for creating centroids
            test_centroids = pre_data[pre_data["test_control_flag"] == "test"]

            #Selecting only the required columns for K-Means for Test
            clustering_data_test = test_centroids[cols]

            #Keeping unique test centroids for clustering
            clustering_data_test = clustering_data_test.drop_duplicates()

            #Total unique test entries
            test_rows = len(clustering_data_test)

            #Keeping Test above Control
            test_d = pre_data[pre_data["test_control_flag"] == "test"]
            control_d = pre_data[pre_data["test_control_flag"] == "control"]
            pre_data = test_d.append(control_d)

            #Selecting only the required columns for K-Means for control
            clustering_data = pre_data[cols]

            #Assigning K-Means parameters
            kmeans = KMeans(n_clusters = test_rows,
                            init = np.array(clustering_data_test),
                            n_init = 1, max_iter = 1, random_state = 42)

            #Selecting only the required columns for K-Means for test
            clustering_data_t = test_d[cols]
            
            ####test_d==clustering_data_test?????

            #Running K-Means for test - all unique test data points will be assigned different clusters
            identified_clusters = kmeans.fit_predict(clustering_data_t)

            #print("K-Means Test done")

            #Merging cluster column with the original dataset which has been filtered on pre period and test
            test_d['clusters'] = identified_clusters

            #Selecting only the required columns for K-Means for control
            clustering_data_c = control_d[cols]

            #Running K-Means for control
            identified_clusters = kmeans.fit_predict(clustering_data_c)

            #print("K-Means done")

            #Merging cluster column with the original dataset which has been filtered on pre period and control
            control_d['clusters'] = identified_clusters

            #Appending clustering results for testa nd control
            pre_data = test_d.append(control_d)

            #Look-alike matching within each cluster
            for k in range(test_rows):

                #print("Cluster: " + str(k))

                clust_0 = pre_data[pre_data["clusters"] == k]

                #Remove control patients who have already been matched---------HOW?
                #Select Control patients from cluster group who are not in control_non_replacement
                clust_0 = clust_0.drop(['_merge'], axis=1)
                clust_0 = pd.merge(clust_0, control_non_replacement, on=["patient_id"], how="outer", 
                                   indicator=True).query('_merge=="left_only"')

                #Total distict test_control_flag
                t_c_len = len(pd.unique(clust_0['test_control_flag']))

                #print (t_c_len)

                #If in a cluster, there is only either test or control patients
                if t_c_len == 1:
                    t_c_value = clust_0.test_control_flag.unique()
                    if t_c_value == ['test']: #Add T to the unmatched list to match with Patient table
                        T_only = clust_0[["patient_id", "combination_key", "clusters"]]
                        test_unmatched_list = test_unmatched_list.append(T_only)

                if t_c_len == 2:
                    #Find distance of each control patient to test patient
                    #Extract Test patient
                    T_only = clust_0[clust_0["test_control_flag"] == "test"]

                    #Extract Control patients
                    C_only = clust_0[clust_0["test_control_flag"] == "control"]

                    #Number of test patients
                    test_count = len(T_only)

                    #Number of control patients
                    control_count = len(C_only)

                    #Check if #test patients > #control patients
                    if control_count < test_count:
                        #Select additional test patients and add them to unmatched test list
                        T_unmatched = T_only.tail(test_count - control_count)
                        T_unmatched = T_unmatched[["patient_id", "combination_key"]]
                        T_unmatched["clusters"] = 0
                        test_unmatched_list = test_unmatched_list.append(T_unmatched)

                        #Select the required test patients
                        T_only = T_only.head(control_count)
                        test_count = len(T_only)

                    #Select only one test patient within the cluster
                    T_only_single = pd.DataFrame(T_only.iloc[[0]])

                    #Create a new distance column within Control to find distance from Test Patient to Control
                    C_only["test_distance"] = pd.DataFrame(scipy.spatial.distance.cdist((C_only[cols]).values,
                                                                                        (T_only_single[cols]).values,
                                                                                         metric='euclidean'))

                    #Sort control patients by distance
                    C_only = C_only.sort_values("test_distance")

                    #Find the shortest control patietns to map
                    C_only_single = C_only.head(test_count)

                    #Keeping only Patient ID
                    C_only_single = pd.DataFrame(C_only_single["patient_id"])

                    #Assign control patients to non-replacement list
                    control_non_replacement = control_non_replacement.append(C_only_single)

                    #Match test and control
                    T_only = T_only[["patient_id", "combination_key", "clusters"]]
                    T_only.rename(columns = {'patient_id':'test_patient_id'}, inplace = True)
                    C_only_single.rename(columns = {'patient_id':'control_patient_id'}, inplace = True)
                    T_only = T_only.reset_index(drop=True)
                    C_only_single = C_only_single.reset_index(drop=True)
                    T_only = T_only.join(C_only_single)
                    matched_list = matched_list.append(T_only)
                    print(matched_list)


        return matched_list, test_unmatched_list, control_non_replacement



    def combination_func (self,comb_data, matched_list, test_unmatched_list, control_non_replacement):

        #Looping over all 6 pre-periods
        for j in range(1, 7):

            #print("Pre-Period: " + str(j))

            pre_data = comb_data[comb_data["pre_window"] == j]
            matched_list, test_unmatched_list, control_non_replacement = self.pre_period_func (pre_data, matched_list,
                                                                                          test_unmatched_list,
                                                                                          control_non_replacement)
        #matched_list.to_csv("./output/matched_list.csv", index=False)
        test_unmatched_list.to_csv("./output/test_unmatched_list.csv", index=False)
        control_non_replacement.to_csv("./output/control_non_replacement.csv", index=False)
        return matched_list, test_unmatched_list, control_non_replacement


    def predict(self,input_data):
        
        print("Entry to predict func")
        #Creating a dataframe to store control patients who have been matched so that they are not matched 
        #again within the same combination group
        control_non_replacement = pd.DataFrame(columns = ["patient_id"])  #######CHECK
        #Ordering Test and Control - This should be handled by DE team
        input_data_test = input_data[input_data["test_control_flag"] == "test"]
        input_data_control = input_data[input_data["test_control_flag"] == "control"]
        input_data = input_data_test.append(input_data_control)

        #Creating a dataframe to store matched list of patients
        matched_list = pd.DataFrame(columns = ["test_patient_id", "combination_key", "clusters", "control_patient_id"])

        #Creating a dataframe to store un-matched test list of patients
        test_unmatched_list = pd.DataFrame(columns = ["patient_id", "combination_key", "clusters"])

        #Total distict combinations
        distict_combinations = input_data.combination_key.unique()
        combination_number = 0

        #Looping over all combinations
        for i in distict_combinations:
            print(i)
            combination_number = combination_number + 1
            comb_data = input_data[input_data["combination_key"] == i]
            matched_list, test_unmatched_list, control_non_replacement = self.combination_func(comb_data, 
                                                                                               matched_list, 
                                                                                               test_unmatched_list, 
                                                                                               control_non_replacement)
        matched_list =matched_list.reset_index()
        #df_final=pd.DataFrame()
        
        #df_final.to_csv("./output/df_final.csv", index=False)
        #return df_final
    

Entry


In [15]:
cluster = Clustering()
df = pd.read_csv("./input/Sample_NC_clustering.csv")
output = cluster.predict(df)

Entry to predict func
18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0
  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   

  control_patient_id  
0                 32  
  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               7  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   

  control_patient_id  
0                 32  
0                 89  
  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               7  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              13  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   

  control_patient_id  
0                 32  
0                 89  
0                 87  
  test_patient_id                             combination_k

  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               7  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              13  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               2  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               3  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0               8  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               9  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        3   
0              14  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        4   
0              15  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        5   
0               4  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0              10  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              16  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               5  18-40_0_NORTH-EAST_

  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               7  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              13  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               2  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               3  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0               8  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               9  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        3   
0              14  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        4   
0              15  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        5   
0               4  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0              10  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              16  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               5  18-40_0_NORTH-EAST_

  test_patient_id                             combination_key clusters  \
0               1  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               7  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              13  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               2  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0               3  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0               8  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               9  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        3   
0              14  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        4   
0              15  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        5   
0               4  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        0   
0              10  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        1   
0              16  18-40_0_NORTH-EAST_0_0_1_0_0_1_1_0_0_0_0_0        2   
0               5  18-40_0_NORTH-EAST_